# Eclat


## Importar las librerías


In [3]:
!pip install apyori

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Preprocesamiento de datos


In [5]:
datos = pd.read_csv("Compras_Tienda.csv", header=None)
datos.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
1,burgers,meatballs,eggs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chutney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,turkey,avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mineral water,milk,energy bar,whole wheat rice,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
len(datos)

7501

In [7]:
transacciones = []
for i in range(0, len(datos)):
    transacciones.append([str(datos.values[i, j]) for j in range(0, 20)])

In [8]:
for i in range(5):
    print(transacciones[i])

['shrimp', 'almonds', 'avocado', 'vegetables mix', 'green grapes', 'whole weat flour', 'yams', 'cottage cheese', 'energy drink', 'tomato juice', 'low fat yogurt', 'green tea', 'honey', 'salad', 'mineral water', 'salmon', 'antioxydant juice', 'frozen smoothie', 'spinach', 'olive oil']
['burgers', 'meatballs', 'eggs', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan']
['chutney', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan']
['turkey', 'avocado', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan']
['mineral water', 'milk', 'energy bar', 'whole wheat rice', 'green tea', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan']


## Entrenar el modelo Eclat con el conjunto de datos

**NOTA:** Se puede ejecutar sin las condiciones de confianza (confidence) y de mejora de confianza (lift) ya que ECLAT no los requiere. Sin embargo, no es mala idea dejarlos ya que nos asegura tener reglas más robustas


In [9]:
from apyori import apriori

reglas = apriori(
    transactions=transacciones,
    min_support=0.003,
    min_confidence=0.2,
    min_lift=3,
    min_length=2,
    max_length=2,
)

## Visualizar los resultados


### Desplegar los primeros resultados provenientes directamente de la función apriori


In [10]:
resultados = list(reglas)

In [11]:
resultados

[RelationRecord(items=frozenset({'chicken', 'light cream'}), support=0.004532728969470737, ordered_statistics=[OrderedStatistic(items_base=frozenset({'light cream'}), items_add=frozenset({'chicken'}), confidence=0.29059829059829057, lift=4.84395061728395)]),
 RelationRecord(items=frozenset({'escalope', 'mushroom cream sauce'}), support=0.005732568990801226, ordered_statistics=[OrderedStatistic(items_base=frozenset({'mushroom cream sauce'}), items_add=frozenset({'escalope'}), confidence=0.3006993006993007, lift=3.790832696715049)]),
 RelationRecord(items=frozenset({'pasta', 'escalope'}), support=0.005865884548726837, ordered_statistics=[OrderedStatistic(items_base=frozenset({'pasta'}), items_add=frozenset({'escalope'}), confidence=0.3728813559322034, lift=4.700811850163794)]),
 RelationRecord(items=frozenset({'honey', 'fromage blanc'}), support=0.003332888948140248, ordered_statistics=[OrderedStatistic(items_base=frozenset({'fromage blanc'}), items_add=frozenset({'honey'}), confidence=0

### Organizar los resultados en un DataFrame de Pandas

**Nota:** acá se removieron los valores de _confidence_ y _lift_ ya que no se utilizan para este modelo. También se modifican los nombres de lado izquierdo y lado derecho


In [12]:
def organizar(resultados):
    lado_izq = [tuple(resultado[2][0][0])[0] for resultado in resultados]
    lado_der = [tuple(resultado[2][0][1])[0] for resultado in resultados]
    supports = [resultado[1] for resultado in resultados]
    return list(zip(lado_izq, lado_der, supports))


df_resultados = pd.DataFrame(
    organizar(resultados), columns=["Producto 1", "Producto 2", "Support"]
)

### Desplegar los resultados en orden descendente de Support


In [13]:
df_resultados.nlargest(n=10, columns="Support")

,Producto 1,Producto 2,Support
4,herb & pepper,ground beef,0.015998
7,whole wheat pasta,olive oil,0.007999
2,pasta,escalope,0.005866
1,mushroom cream sauce,escalope,0.005733
5,tomato sauce,ground beef,0.005333
8,pasta,shrimp,0.005066
0,light cream,chicken,0.004533
3,fromage blanc,honey,0.003333
6,light cream,olive oil,0.003200


Si se quere trabajar con conjuntos mayores de 2 elementos, solo es necesario modificar el parámetro "max_length" en la celda de entrenamiento del modelo. Este último reporte seguirá saiendo con dos columnas Producto 1 y Producto 2, pero los productos adicionales saldrán en la columna Producto 1.
